In [18]:
from datasets import load_dataset
import pandas as pd
import spacy as sp
from spacy.lang.en.stop_words import STOP_WORDS

import re
import nltk
from nltk.corpus import stopwords
from gensim import corpora
from gensim.models import LdaModel
from sklearn.feature_extraction.text import TfidfVectorizer
import pyLDAvis.gensim
import pyLDAvis
from nltk.stem import WordNetLemmatizer

In [19]:
lemmatizer = WordNetLemmatizer()

In [2]:
# view the col context fully...

pd.set_option('display.max_colwidth', None)

In [3]:
ds = load_dataset("Amod/mental_health_counseling_conversations")

In [4]:
df = ds['train'].to_pandas()

In [5]:
unique_contexts = df['Context'].drop_duplicates()

In [8]:
len(conversations)

995

In [21]:
# Download stopwords if you haven't already
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shafayetulislam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# Create a dictionary representation of the conversations
dictionary = corpora.Dictionary(processed_conversations)

# Create a bag-of-words representation of each conversation
corpus = [dictionary.doc2bow(text) for text in processed_conversations]

In [24]:
def preprocess_text(text):
    # Remove punctuation, lowercase, and remove stopwords
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text.lower())  # Remove punctuation and lowercase
    tokens = text.split()  # Tokenize by space
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    
    return tokens

# Apply preprocessing to each conversation
processed_conversations = [preprocess_text(convo) for convo in conversations]

In [7]:
conversations = unique_contexts.tolist()

In [16]:
# Define the number of topics
num_topics = 3

# Create LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42, passes=10, alpha='auto')

# Display the top words in each topic
for i, topic in lda_model.print_topics(num_words=5):
    print(f"Topic {i}: {topic}")

Topic 0: 0.020*"im" + 0.014*"dont" + 0.012*"feel" + 0.011*"want" + 0.010*"like"
Topic 1: 0.023*"im" + 0.014*"dont" + 0.011*"feel" + 0.011*"know" + 0.011*"get"
Topic 2: 0.014*"feel" + 0.013*"like" + 0.010*"im" + 0.010*"time" + 0.009*"dont"


In [17]:
# Prepare visualization (pyLDAvis)
lda_display = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [10]:
# iloc is used to get specific index position of this dataframe. 

print(unique_contexts.iloc[5])

I have a severe back problem. I've had 3 major and several minor operations, but I'm still in constant pain. How can I deal with the depression from this chronic pain?


In [15]:
# spacy understanding
nlp = sp.load('en_core_web_sm')
doc = nlp('This is nlp with spaCy')

In [31]:
# making our own stop words
stop_words = ['say', 'said', 'it', 'an', 'none', 'all', 'saying']
for stopwords in stop_words:
  lexeme = nlp.vocab[stopwords]
  lexeme.is_stop = True

In [28]:
# spacy's own stop word dictionary
len(STOP_WORDS)

328

In [29]:
# adding our own stopword
for word in stop_words:
    STOP_WORDS.add(word)

In [30]:
len(STOP_WORDS)

328

In [32]:
# lemmatization practice
doc = nlp('Real-time training during global emergencies is critical for effective preparedness and response.')

sentence = []
for w in doc:
  # if it’s not a stop word or punctuation mark, add it to our article!
  if w.text != 'n' and not w.is_stop and not w.is_punct and not w.like_num:
    # we add the lemmatized version of the word
    sentence.append(w.lemma_)
print(sentence)

['real', 'time', 'training', 'global', 'emergency', 'critical', 'effective', 'preparedness', 'response']


In [ ]:
# Vectorizing text and transformations and n-grams